In [1]:
from datasets import load_dataset

news_datasets = load_dataset("indic_glue","bbca.hi")

Found cached dataset indic_glue (C:/Users/arifa/.cache/huggingface/datasets/indic_glue/bbca.hi/1.0.0/082910ba488a202e0b57bfff0222f736130a17028e9ff242e4d537bd2fe6cd95)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
news_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 3467
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 866
    })
})

In [3]:
news_datasets["train"].set_format("pandas")

In [4]:
# get label counts for both classes
label_counts = news_datasets["train"]["label"].value_counts()
num_labels = (len(label_counts.keys()))

In [5]:
label_counts

label
india              1389
international       904
entertainment       285
sport               258
news                230
science             194
business             54
pakistan             43
southasia            42
institutional        19
social               18
china                14
multimedia           12
learningenglish       5
Name: count, dtype: int64

In [6]:
news_datasets["train"].reset_format()

In [7]:
from transformers import set_seed

# set_seed(30)
set_seed(42)

In [8]:
from transformers import CharacterBertTokenizer, AutoModelForSequenceClassification

tokenizer = CharacterBertTokenizer(strip_accents=None, do_lower_case=None)

In [9]:
from transformers import BertForSequenceClassification, BertConfig, CharacterBertModel, CharacterBertTokenizer

#### LOADING BERT FOR CLASSIFICATION ####

config = BertConfig.from_pretrained('bert-base-uncased', num_labels=num_labels)  # binary classification
model = BertForSequenceClassification(config=config)

In [10]:
#### REPLACING BERT WITH CHARACTER_BERT ####

character_bert_model = CharacterBertModel.from_pretrained(
    "E:\Documents\Character Bert\Hate Speech\character-bert-hindi")
model.bert = character_bert_model

Some weights of the model checkpoint at E:\Documents\Character Bert\Hate Speech\character-bert-hindi were not used when initializing CharacterBertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing CharacterBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CharacterBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def tokenize_function(example):
    #return tokenizer(example["text"], truncation=True)
    return tokenizer(example["text"], truncation=True, max_length=128)

In [12]:
from transformers import DataCollatorWithPadding

tokenized_datasets = news_datasets.map(tokenize_function, batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at C:\Users\arifa\.cache\huggingface\datasets\indic_glue\bbca.hi\1.0.0\082910ba488a202e0b57bfff0222f736130a17028e9ff242e4d537bd2fe6cd95\cache-d88d59429e7ba57b.arrow
Loading cached processed dataset at C:\Users\arifa\.cache\huggingface\datasets\indic_glue\bbca.hi\1.0.0\082910ba488a202e0b57bfff0222f736130a17028e9ff242e4d537bd2fe6cd95\cache-8c0e741a951db135.arrow


In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3467
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 866
    })
})

In [14]:
# "india": 0,
# "international": 1,
# "entertainment": 2,
# "sport":3,
# "news": 4,
# "science": 5,
# "business": 6,
# "pakistan": 7,
# "southasia":8,
# "institutional":  9,
# "social":10,
# "china": 11,
# "multimedia":  12,
# "learningenglish": 13

In [15]:
def assign_label(example):
    mapping = {
        "india": 0,
        "international": 1,
        "entertainment": 2,
        "sport":3,
        "news": 4,
        "science": 5,
        "business": 6,
        "pakistan": 7,
        "southasia":8,
        "institutional":  9,
        "social":10,
        "china": 11,
        "multimedia":  12,
        "learningenglish": 13
    }
    example['label'] = mapping[example['label']]
    return example

In [16]:
tokenized_datasets = tokenized_datasets.map(assign_label)
tokenized_datasets.set_format("torch")
tokenized_datasets.column_names

Loading cached processed dataset at C:\Users\arifa\.cache\huggingface\datasets\indic_glue\bbca.hi\1.0.0\082910ba488a202e0b57bfff0222f736130a17028e9ff242e4d537bd2fe6cd95\cache-c4601a3e57037428.arrow
Loading cached processed dataset at C:\Users\arifa\.cache\huggingface\datasets\indic_glue\bbca.hi\1.0.0\082910ba488a202e0b57bfff0222f736130a17028e9ff242e4d537bd2fe6cd95\cache-9344ad9a11970411.arrow


{'train': ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
 'test': ['label', 'input_ids', 'token_type_ids', 'attention_mask']}

In [17]:
samples = [tokenized_datasets["train"][i] for i in range(5)]
samples

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] मेट्रो की इस लाइन के चलने से दक्षिणी दिल्ली से नोएडा जाने का समय काफी कम हो जाएगा और यात्रियों को राजीव चौक या मंडी हाउस से होकर नहीं जाना पड़ेगा. लेकिन, यह मजेंटा लाइन इसलिए भी महत्वपूर्ण है क्योंकि इस पर ड्राइवलैस यानी बिना ड्राइवर वाली मेट्रो चलाने की योजना है. ऐसा भारत में पहली बार होगा जब कोई मेट्रो बिना ड्राइवर के चलाई जाएगी. मेट्रो के तीसरे फेज में भारत में पहली बार ड्राइवरलेस तकनीक आएगी लेकिन दुनिया भर में कई देशों में ड्राइवरलेस मेट्रो पहले से ही चलती हैं. इन देशों में ड्राइवरलेस मेट्रो सफल भी रही हैं. दक्षिण कोरिया की राजधानी सोल में ड्राइवलेस ट्रेन कामयाबी से चल रही है. ये मेट्रो ज़मीन के नीचे [SEP]'

'>>> [CLS] नेटिजन यानि इंटरनेट पर सक्रिय नागरिक अब ट्विटर पर सरकार द्वारा लगाए प्रतिबंधों के समर्थन या विरोध में अपने विचार व्यक्त करते हैं और वेबसाइट संबंधी सूचना भी जारी करते हैं. कुछ दिन पहले ही शियाओं के खिलाफ़ हिंसा पर नज़र रखनेवाली एक प्रतिबंधित वेबसाइट शियाकिलिंग. कॉम को ट्विटर पर चलाए अभियान और सड़कों पर हुए प्रदर्शन के बाद बहाल कर दिया गया था. उससे कुछ दिन 

In [18]:
news_datasets["train"][:5]

{'label': ['india', 'pakistan', 'news', 'india', 'india'],
 'text': ['मेट्रो की इस लाइन के चलने से दक्षिणी दिल्ली से नोएडा जाने का समय काफी कम हो जाएगा और यात्रियों को राजीव चौक या मंडी हाउस से होकर नहीं जाना पड़ेगा.लेकिन, यह मजेंटा लाइन इसलिए भी महत्वपूर्ण है क्योंकि इस पर ड्राइवलैस यानी बिना ड्राइवर वाली मेट्रो चलाने की योजना है. ऐसा भारत में पहली बार होगा जब कोई मेट्रो बिना ड्राइवर के चलाई जाएगी. मेट्रो के तीसरे फेज में भारत में पहली बार ड्राइवरलेस तकनीक आएगी लेकिन दुनिया भर में कई देशों में ड्राइवरलेस मेट्रो पहले से ही चलती हैं. इन देशों में ड्राइवरलेस मेट्रो सफल भी रही हैं. दक्षिण कोरिया की राजधानी सोल में ड्राइवलेस ट्रेन कामयाबी से चल रही है. ये मेट्रो ज़मीन के नीचे चलती है इसमें ड्राइवर का केबिन भी नहीं होता है. यूरोप में डेनमार्क, स्पेन, इटली, फ्रांस, जर्मनी, हंगरी, स्विट्जरलैंड और ब्रिटेन में भी ड्राइवरलैस मेट्रो चलती है. इन देशों में एक से ज़्यादा शहरों में भी ऐसी मेट्रो चलाई जाती है. इनके अलावा अमेरिका और कनाडा में भी ड्राइवरलेस ट्रेन चलती है. वहीं, ब्राजील, पेरू और चाइल में

In [19]:
from torch.utils.data import DataLoader
# batch_size = 16
batch_size = 32

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=batch_size, collate_fn=data_collator
)

In [20]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([32, 128, 50]),
 'token_type_ids': torch.Size([32, 128]),
 'attention_mask': torch.Size([32, 128]),
 'labels': torch.Size([32])}

In [21]:
import torch
with torch.no_grad():
    outputs = model(**batch)
    print(outputs.loss, outputs.logits.shape)

tensor(2.6730) torch.Size([32, 14])


In [22]:
import numpy as np
import evaluate

metric_fun = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    metric_result = metric_fun.compute(references=labels, predictions=predictions)
    return {
        "accuracy": metric_result["accuracy"],
    }

In [23]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [24]:
from transformers import TrainingArguments

# batch_size = 16
batch_size = 32
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size


training_args = TrainingArguments(
    report_to = None,
    output_dir="models/bbc-classifier",
    overwrite_output_dir=True,
    save_strategy="no",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    #learning_rate=3e-5,
    weight_decay=0.01,
    #weight_decay=0.02,
    #warmup_ratio=0.05,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    #num_train_epochs=4,
    #push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [25]:
# from datasets import concatenate_datasets

# entire_train = concatenate_datasets([tokenized_datasets["train"], tokenized_datasets["validation"]]) 

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    #train_dataset=entire_train,
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using amp half precision backend


In [27]:
trainer.train()

***** Running training *****
  Num examples = 3467
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 654


Epoch,Training Loss,Validation Loss,Accuracy
1,1.252800,0.993218,0.700924
2,0.897700,0.878949,0.751732
3,0.753700,0.845290,0.758661
4,0.641300,0.806878,0.760970
5,0.530600,0.822459,0.759815
6,0.452700,0.837284,0.760970


***** Running Evaluation *****
  Num examples = 866
  Batch size = 32
***** Running Evaluation *****
  Num examples = 866
  Batch size = 32
***** Running Evaluation *****
  Num examples = 866
  Batch size = 32
***** Running Evaluation *****
  Num examples = 866
  Batch size = 32
***** Running Evaluation *****
  Num examples = 866
  Batch size = 32
***** Running Evaluation *****
  Num examples = 866
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=654, training_loss=0.7522476714685422, metrics={'train_runtime': 299.44, 'train_samples_per_second': 69.47, 'train_steps_per_second': 2.184, 'total_flos': 8.32504219679232e+16, 'train_loss': 0.7522476714685422, 'epoch': 6.0})

In [28]:
trainer.evaluate(tokenized_datasets["test"])

***** Running Evaluation *****
  Num examples = 866
  Batch size = 32


{'eval_loss': 0.8372840881347656,
 'eval_accuracy': 0.7609699769053118,
 'eval_runtime': 5.3251,
 'eval_samples_per_second': 162.626,
 'eval_steps_per_second': 5.258,
 'epoch': 6.0}

In [75]:
trainer.save_model()

In [31]:
model = AutoModelForSequenceClassification.from_pretrained("models/bert-unigram-bengali-classifier")
model.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [37]:
trainer.evaluate(tokenized_datasets["validation"])

{'eval_loss': 0.47230616211891174,
 'eval_accuracy': 0.9029057406094968,
 'eval_runtime': 3.4463,
 'eval_samples_per_second': 409.426,
 'eval_steps_per_second': 13.058,
 'epoch': 6.0}